# RSI model-02
https://chatgpt.com/c/67fc9738-fe44-800a-8311-114bb4f82b3b

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from ta.momentum import RSIIndicator
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns


In [ ]:
# Load full training data
df = pd.read_csv("XAGUSD-H1-rates.csv", sep=r'\s+', engine='python')
df.columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'TickVol', 'Vol', 'Spread']

# Convert to datetime
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.sort_values('Datetime')
df.reset_index(drop=True, inplace=True)

# Use RSIIndicator from ta
rsi = RSIIndicator(close=df['Close'], window=14)
df['RSI'] = rsi.rsi()

# Drop NaN rows from RSI calculation
df.dropna(inplace=True)

In [ ]:
# Labeling reversal (RSI 30/70)
def label_reversal_targets(data, future_window=5):
    labels = []
    for i in range(len(data) - future_window):
        rsi_now = data.iloc[i]['RSI']
        rsi_future = data.iloc[i+1:i+future_window+1]['RSI']

        if rsi_now < 30 and any(r > 35 for r in rsi_future):
            labels.append(1)  # Bullish reversal
        elif rsi_now > 70 and any(r < 65 for r in rsi_future):
            labels.append(1)  # Bearish reversal
        else:
            labels.append(0)
    return np.array(labels)

labels = label_reversal_targets(df)


In [ ]:
# Prepare input windows
SEQ_LEN = 30
features = ['Open', 'High', 'Low', 'Close', 'RSI']

X = []
for i in range(len(df) - SEQ_LEN - 5):  # 5 for future reversal window
    window = df.iloc[i:i+SEQ_LEN][features].values
    X.append(window)

X = np.array(X)
y = labels[:len(X)]

print("X shape:", X.shape)
print("y shape:", y.shape)


In [ ]:
# Train/Test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

# LSTM model
from tensorflow.keras.utils import to_categorical
# Don't one-hot encode labels if you use sparse_categorical_crossentropy
# y = to_categorical(y) ← not needed here

model = Sequential([
    LSTM(64, input_shape=(SEQ_LEN, len(features))),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes
])

model.compile(optimizer=Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training
history = model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_val, y_val))


In [ ]:
# Predict on rows1-30.csv
df_input = pd.read_csv("new-data-for-test/rows-30-from-20240503.csv", sep=r'\s+', engine='python')
df_input.columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'TickVol', 'Vol', 'Spread']


# Use RSIIndicator
rsi_input = RSIIndicator(close=df_input['Close'], window=14)
df_input['RSI'] = rsi_input.rsi()

df_input.dropna(inplace=True)  # Drop NaNs from RSI

# Build input sample
sample = df_input[['Open', 'High', 'Low', 'Close', 'RSI']].values[-30:]
sample = np.expand_dims(sample, axis=0)  # Shape: (1, 30, 5)


# Predict
predictions = model.predict(sample)  # shape: (1, 3)
pred_class = np.argmax(predictions[0])
pred_prob = predictions[0][pred_class]

print("Reversal classification:", ["No reversal", "Bullish reversal", "Bearish reversal"][pred_class])
print("Confidence:", round(pred_prob * 100, 2), "%")

if pred_class == 1:
    print("📈 Bullish reversal likely – consider buying!")
elif pred_class == 2:
    print("📉 Bearish reversal likely – consider selling!")
else:
    print("✅ No clear reversal – hold position.")

In [ ]:
# Convert 'Date' and 'Time' to datetime for plotting
df_input['Datetime'] = pd.to_datetime(df_input['Date'] + ' ' + df_input['Time'])

# Set seaborn style
sns.set(style="darkgrid")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True, gridspec_kw={'height_ratios': [3, 1]})

# --- Price Chart (Open & Close) ---
ax1.plot(df_input['Datetime'], df_input['Open'], label='Open Price', color='skyblue', marker='o')
ax1.plot(df_input['Datetime'], df_input['Close'], label='Close Price', color='orange', marker='o')
ax1.set_ylabel("Price")
ax1.set_title("Last 30 Candles (Open & Close)")
ax1.legend()

# --- RSI Chart ---
ax2.plot(df_input['Datetime'], df_input['RSI'], label='RSI', color='purple', marker='.')
ax2.axhline(70, color='red', linestyle='--', label='Overbought (70)')
ax2.axhline(30, color='green', linestyle='--', label='Oversold (30)')
ax2.set_ylabel("RSI")
ax2.set_title("RSI Indicator")
ax2.legend()

# --- Mark Predicted Reversal ---
pred_time = df_input['Datetime'].iloc[-1]  # Last candle
if pred_class == 1:
    ax1.axvline(pred_time, color='green', linestyle='--', lw=2)
    ax1.text(pred_time, df_input['High'].max(), 'Bullish Reversal', color='green', fontsize=12, ha='left')
    ax2.axvline(pred_time, color='green', linestyle='--', lw=2)
elif pred_class == 2:
    ax1.axvline(pred_time, color='red', linestyle='--', lw=2)
    ax1.text(pred_time, df_input['High'].max(), 'Bearish Reversal', color='red', fontsize=12, ha='left')
    ax2.axvline(pred_time, color='red', linestyle='--', lw=2)
else:
    ax1.axvline(pred_time, color='gray', linestyle='--', lw=2)
    ax1.text(pred_time, df_input['High'].max(), 'No Reversal', color='gray', fontsize=12, ha='left')
    ax2.axvline(pred_time, color='gray', linestyle='--', lw=2)

# Format x-axis
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()